In [2]:
#%matplotlib widget
#from backtrader import plot
import backtrader as bt
#import backtrader.analyzers as btanalyzers
import rba_tools as rba
import pandas as pd
import numpy as np
#import matplotlib
from datetime import datetime,date,timedelta
from backtrader.utils import num2date,date2num
import rba_tools.backtest.rba_backtrader_set as rbs
import plotly.graph_objects as go
import plotly.express as px
import rba_tools.backtest.backtrader_extensions.plotting.plotinfo as rbsplotinfo
import rba_tools.backtest.backtrader_extensions.plotting.plot as rbsplot
import importlib
from dataclasses import dataclass,field,asdict,is_dataclass

In [3]:
importlib.reload(rbsplotinfo)
importlib.reload(rbsplot)

<module 'rba_tools.backtest.backtrader_extensions.plotting.plot' from '/home/avery/Documents/rba_tools/src/rba_tools/backtest/backtrader_extensions/plotting/plot.py'>

In [4]:
class MaCrossStrategy(bt.Strategy):

    def __init__(self):
        ma_fast = bt.ind.SMA(period = 10)
        ma_slow = bt.ind.SMA(period = 50)

        self.crossover = bt.ind.CrossOver(ma_fast, ma_slow)

    def next(self):
        if not self.position:
            if self.crossover > 0:
                self.buy()
        elif self.crossover < 0:
            self.close()

In [5]:
cerebro = bt.Cerebro()
puller = rba.retriever.get_crypto_data.DataPuller.kraken_puller()
symbol = 'ETH/USD'
from_date = '11/1/20'
to_date = '12/31/20'

In [6]:
dataframe = puller.fetch_df(symbol, 'h', from_date, to_date)
data = bt.feeds.PandasData(dataname=dataframe,
                               # datetime='Date',
                               nocase=True,
                               )
cerebro.adddata(data)
cerebro.addstrategy(MaCrossStrategy)
cerebro.broker.setcash(1000.0)
cerebro.addsizer(bt.sizers.PercentSizer, percents = 10)

In [7]:
back = cerebro.run()

In [8]:
strategy = back[0]

In [9]:
st_dtime = strategy.lines.datetime.plot()

In [10]:
mylist = strategy.getobservers()
mylist

In [11]:
buysell = mylist[1]
buysell

In [12]:
ind = mylist[0]
indicator = ind

In [13]:
lineidx = 0
line = ind.lines[lineidx]
linealias = ind.lines._getlinealias(lineidx)

In [14]:
line

In [15]:
vals = line.plotrange(0,len(line))

In [16]:
np.arange(5)

array([0, 1, 2, 3, 4])

In [17]:
np.array(vals)

array([1000.        , 1000.        , 1000.        , ..., 1066.95710236,
       1066.95710236, 1066.95710236])

In [18]:
fig = go.Figure(data=go.Scatter(x=np.arange(len(line)),y=np.array(vals)))
fig.show()

In [19]:
inf0 = rbsplotinfo.get_line_plot_info_from_indicator_line(buysell, 0)
inf1 = rbsplotinfo.get_line_plot_info_from_indicator_line(buysell, 1)
inf0

OrderedDict([('marker', '^'),
             ('markersize', 8.0),
             ('color', 'lime'),
             ('fillstyle', 'full'),
             ('ls', '')])

In [20]:
dpi = rbsplotinfo.DataAndPlotInfoContainer(strategy)
dpi

DataAndPlotInfoContainer(strategy='MaCrossStrategy', data_and_plots_list=[DataPlotInfo(df=                       Open    High     Low   Close       Volume  \
2020-11-01 00:00:00  386.62  391.00  385.13  387.33  2572.468658   
2020-11-01 01:00:00  387.33  390.41  387.32  389.00  1682.308921   
2020-11-01 02:00:00  389.05  389.77  387.80  387.91   264.686466   
2020-11-01 03:00:00  387.48  388.33  386.35  386.99   296.637685   
2020-11-01 04:00:00  386.60  386.79  385.07  386.63   944.605609   
...                     ...     ...     ...     ...          ...   
2020-12-31 19:00:00  740.83  742.97  738.44  739.53  1559.446486   
2020-12-31 20:00:00  739.46  745.37  739.07  745.09  1365.984380   
2020-12-31 21:00:00  745.38  745.68  737.49  739.08  1334.269917   
2020-12-31 22:00:00  739.27  743.00  736.68  741.50  1388.991292   
2020-12-31 23:00:00  741.39  742.34  733.28  737.45  1547.984986   

                     cash (fund=None)  value (fund=None)  \
2020-11-01 00:00:00       1000.00

In [21]:
dpi.strategy

'MaCrossStrategy'

In [22]:
dap = dpi.data_and_plots_list[0]
dap

DataPlotInfo(df=                       Open    High     Low   Close       Volume  \
2020-11-01 00:00:00  386.62  391.00  385.13  387.33  2572.468658   
2020-11-01 01:00:00  387.33  390.41  387.32  389.00  1682.308921   
2020-11-01 02:00:00  389.05  389.77  387.80  387.91   264.686466   
2020-11-01 03:00:00  387.48  388.33  386.35  386.99   296.637685   
2020-11-01 04:00:00  386.60  386.79  385.07  386.63   944.605609   
...                     ...     ...     ...     ...          ...   
2020-12-31 19:00:00  740.83  742.97  738.44  739.53  1559.446486   
2020-12-31 20:00:00  739.46  745.37  739.07  745.09  1365.984380   
2020-12-31 21:00:00  745.38  745.68  737.49  739.08  1334.269917   
2020-12-31 22:00:00  739.27  743.00  736.68  741.50  1388.991292   
2020-12-31 23:00:00  741.39  742.34  733.28  737.45  1547.984986   

                     cash (fund=None)  value (fund=None)  \
2020-11-01 00:00:00       1000.000000        1000.000000   
2020-11-01 01:00:00       1000.000000        10

In [23]:
key = 'Broker'
broker_ind = dap.indicator_list[0]
broker_ind

IndicatorPlotInfo(name='Broker', line_list=[LinePlotInfo(line_name='cash (fund=None)', plotinfo={}, markers={}, mode=None), LinePlotInfo(line_name='value (fund=None)', plotinfo={}, markers={}, mode=None)])

In [24]:
myfig = rbsplot.plot_indicator(dpi.data_and_plots_list[0].df, broker_ind)
myfig.show()

In [25]:
test = strategy.data0._dataname
test

,Open,High,Low,Close,Volume,Symbol
Timestamp,,,,,,
2020-11-01 00:00:00,386.62,391.00,385.13,387.33,2572.468658,ETH/USD
2020-11-01 01:00:00,387.33,390.41,387.32,389.00,1682.308921,ETH/USD
2020-11-01 02:00:00,389.05,389.77,387.80,387.91,264.686466,ETH/USD
2020-11-01 03:00:00,387.48,388.33,386.35,386.99,296.637685,ETH/USD
2020-11-01 04:00:00,386.60,386.79,385.07,386.63,944.605609,ETH/USD
...,...,...,...,...,...,...
2020-12-31 19:00:00,740.83,742.97,738.44,739.53,1559.446486,ETH/USD
2020-12-31 20:00:00,739.46,745.37,739.07,745.09,1365.984380,ETH/USD
2020-12-31 21:00:00,745.38,745.68,737.49,739.08,1334.269917,ETH/USD


In [26]:
broker_ind

IndicatorPlotInfo(name='Broker', line_list=[LinePlotInfo(line_name='cash (fund=None)', plotinfo={}, markers={}, mode=None), LinePlotInfo(line_name='value (fund=None)', plotinfo={}, markers={}, mode=None)])

In [27]:
line_1 = broker_ind.line_list[0]
line_1

LinePlotInfo(line_name='cash (fund=None)', plotinfo={}, markers={}, mode=None)

In [28]:
rbsplotinfo.pickle_dpic(dpi)

In [29]:
unpickle = rbsplotinfo.unpickle_last_dpic()
unpickle

DataAndPlotInfoContainer(strategy='MaCrossStrategy', data_and_plots_list=[DataPlotInfo(df=                       Open    High     Low   Close       Volume  \
2020-11-01 00:00:00  386.62  391.00  385.13  387.33  2572.468658   
2020-11-01 01:00:00  387.33  390.41  387.32  389.00  1682.308921   
2020-11-01 02:00:00  389.05  389.77  387.80  387.91   264.686466   
2020-11-01 03:00:00  387.48  388.33  386.35  386.99   296.637685   
2020-11-01 04:00:00  386.60  386.79  385.07  386.63   944.605609   
...                     ...     ...     ...     ...          ...   
2020-12-31 19:00:00  740.83  742.97  738.44  739.53  1559.446486   
2020-12-31 20:00:00  739.46  745.37  739.07  745.09  1365.984380   
2020-12-31 21:00:00  745.38  745.68  737.49  739.08  1334.269917   
2020-12-31 22:00:00  739.27  743.00  736.68  741.50  1388.991292   
2020-12-31 23:00:00  741.39  742.34  733.28  737.45  1547.984986   

                     cash (fund=None)  value (fund=None)  \
2020-11-01 00:00:00       1000.00

In [30]:
rbsplot.plot_indicator(unpickle.data_and_plots_list[0].df, unpickle.data_and_plots_list[0].indicator_list[0])